# Amazon EventBridge

<img src="../_assets/aws_service_icons/eventbridge.svg" width="80" alt="Amazon EventBridge">

## Goals
- Understand what EventBridge is
- Know what it’s practically used for
- See a simple **SDK pseudo-code** example (not executed)


## What is EventBridge?

**Amazon EventBridge** is a serverless **event bus** service that routes events between AWS services, your own applications, and SaaS providers.

- Producers publish **events** (JSON payloads) to an **event bus**.
- **Rules** match events using **event patterns** (filtering) or **schedules**.
- Matched events are delivered to **targets** like Lambda, Step Functions, SQS/SNS, Kinesis, and more.

EventBridge is the successor to CloudWatch Events (most event routing features live in EventBridge today).


## What is EventBridge used for (practically)?

EventBridge is commonly used for **decoupling** systems and building **event-driven** architectures (publish events once, let multiple consumers react).

Typical use cases:
- **Event-driven microservices** (e.g., emit `OrderCreated`, `PaymentFailed`, etc.)
- **Workflow triggers** (start Step Functions / Glue jobs / ETL / SageMaker pipelines when data arrives)
- **Scheduling** (cron/rate rules for periodic jobs)
- **Notifications & automation** (route important state changes to SNS/Slack via Lambda)
- **Cross-account routing** to centralize event processing and governance

For ML specifically, EventBridge is often used to trigger feature pipelines, batch inference, retraining jobs, and monitoring alerts.


## Core concepts (minimum you should know)

- **Event bus**: default bus, custom buses, and partner buses
- **Event**: JSON with fields like `source`, `detail-type`, `detail`, `time`, etc.
- **Rule**: an event pattern or schedule that selects events
- **Target**: where matching events are delivered (Lambda, Step Functions, SQS, SNS, Kinesis, etc.)
- **Permissions**: EventBridge must be allowed to invoke the target (resource policies/IAM)
- **Archives & replay** (optional): store events and replay later for backfills


## Using EventBridge with an SDK (pseudo-code)
This is **illustrative pseudo-code** showing a typical flow in an AWS SDK (e.g., Python `boto3`, JS v3, Java, etc.).
**Note**: requires AWS credentials/permissions, a region, and real resource ARNs; add retries/logging/error handling in production.
```python
# PSEUDO-CODE (not executed)
sdk = AwsSdk()
events = sdk.eventbridge(region="us-east-1")
bus_name = "custom-app-bus"  # or use the default bus
rule_name = "route-order-created"
# 1) (One-time) Create a custom event bus (optional)
events.create_event_bus(Name=bus_name)
# 2) Create a rule that matches specific events
# Example event shape:
# {
#   "source": "app.orders",
#   "detail-type": "OrderCreated",
#   "detail": {"orderId": "o-123", "customerId": "c-456"}
# }
events.put_rule(
    Name=rule_name,
    EventBusName=bus_name,
    EventPattern={
        "source": ["app.orders"],
        "detail-type": ["OrderCreated"],
    },
    State="ENABLED",
)
# 3) Add a target (e.g., a Lambda function)
lambda_arn = "arn:aws:lambda:us-east-1:123456789012:function:process-order"
events.put_targets(
    Rule=rule_name,
    EventBusName=bus_name,
    Targets=[
        {
            "Id": "processOrderLambda",
            "Arn": lambda_arn,
            # Optional: transform/reshape payload
            # "InputTransformer": { ... }
            # Optional: send failures to a DLQ
            # "DeadLetterConfig": {"Arn": dlq_arn}
        }
    ],
)
# 4) Emit an event (from a producer service)
events.put_events(
    Entries=[
        {
            "EventBusName": bus_name,
            "Source": "app.orders",
            "DetailType": "OrderCreated",
            "Detail": json_string({"orderId": "o-123", "customerId": "c-456"}),
        }
    ]
)
# 5) The rule matches -> the target is invoked asynchronously
```


## Pitfalls & quick tips

- Design stable `source` and `detail-type` values; treat them like a contract.
- Keep `detail` payloads small; put large blobs in S3 and send references.
- Build consumers to be **idempotent** (event systems often deliver at-least-once).
- Use **DLQs/retries** where available so failures don’t silently drop events.

## References
- AWS Docs: Amazon EventBridge
- AWS Docs: Event patterns and rules
- AWS SDK docs: `PutRule`, `PutTargets`, `PutEvents`
